## Recap курса "Введение в машинное обучение"

## Семинар #10

### Екатерина Кондратьева

ekaterina.kondrateva@skoltech.ru

  Программа курса:
  
   1. Обучение с учителем: Линейная и логистическая регрессия. Ядра.
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/02_lr
    
 
    
   2. Обучение с учителем: Регуляризация в линейных моделях. Метод Ближайших Соседей (KNN)
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/blob/master/03_knn
    

    
   3. Обучение с учителем: Метод опорных векторов (SVM) для задач классификации и регрессии. Kernel SVM
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/04_svc
     

    
   4. Обучение с учителем: Деревья решений (Decision Trees). Случайный лес (Random Forest).
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/05_rfc
    

    
   5. Оценка качества алгоритмов машинного обучения. Кросс-валидация. Поиск аномалий и артефактов в выборке. 
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/06_model_validaton
    

    
   6. Обучение без учителя: кластеризация. Снижение размерности данных PCA.
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/07_clustering_pca
    

    
   7. Отбор и генерация признаков (Feature Engineering). Поиск и оптимизация модели (Grid Search).
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/08_feature_engineering
    

    
   8. Стекинг, вотинг. Градиентный бустинг. Пакеты XGBoost/Catboost/LightGBM  
    
    Фолдер: https://github.com/kondratevakate/machine-learning-with-love/tree/master/09_gradient_boosting
    
    

### Построим пайплан для датасета HousePricing:

Данные - https://www.kaggle.com/lider123/housepricing

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [3]:
train_data = pd.read_csv("housepricing/train.csv")
test_data = pd.read_csv("housepricing/test.csv")

In [4]:
full_data = pd.concat([train_data,test_data])

In [5]:
full_data.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706.0,0.0,...,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,NaN,3,1Fam,TA,Gd,978.0,0.0,...,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007
2,920,866,0,NaN,3,1Fam,TA,Mn,486.0,0.0,...,WD,0,Pave,6,920.0,AllPub,0,2001,2002,2008
3,961,756,0,NaN,3,1Fam,Gd,No,216.0,0.0,...,WD,0,Pave,7,756.0,AllPub,0,1915,1970,2006
4,1145,1053,0,NaN,4,1Fam,TA,Av,655.0,0.0,...,WD,0,Pave,9,1145.0,AllPub,192,2000,2000,2008


In [6]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
Alley            198 non-null object
BedroomAbvGr     2919 non-null int64
BldgType         2919 non-null object
BsmtCond         2837 non-null object
BsmtExposure     2837 non-null object
BsmtFinSF1       2918 non-null float64
BsmtFinSF2       2918 non-null float64
BsmtFinType1     2840 non-null object
BsmtFinType2     2839 non-null object
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtQual         2838 non-null object
BsmtUnfSF        2918 non-null float64
CentralAir       2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
Electrical       2918 non-null object
EnclosedPorch    2919 non-null int64
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      291

In [7]:
categorial_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name == 'object']
numeric_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name != 'object']
print(categorial_features, numeric_features, sep = '\n')

['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape', 'MSZoning', 'MasVnrType', 'MiscFeature', 'Neighborhood', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType', 'Street', 'Utilities']
['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'Id', 'KitchenAbvGr', 'LotArea', 'LotFrontage', 'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal', 'MoSold', 'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'SalePrice',

In [8]:
# заполним пропуски числовых данных применяя свои знания предметной области
full_data["SaleType"].fillna(full_data["SaleType"].mode()[0], inplace=True)
full_data["Electrical"].fillna(full_data["Electrical"].mode()[0], inplace=True)
full_data["MSZoning"].fillna(full_data["MSZoning"].mode()[0], inplace=True)
full_data["GarageArea"].fillna(full_data["GarageArea"].median(), inplace=True)
full_data["CentralAir"] = full_data["CentralAir"].map({"Y": 1, "N": 0})
full_data["LotFrontage"].fillna(full_data["LotFrontage"].median(), inplace=True)
full_data["Alley"].fillna("None", inplace=True)
full_data["MasVnrType"].fillna("None", inplace=True)
full_data["MasVnrArea"].fillna(0, inplace=True)

In [9]:
#заполним пропуски числовых данных подряд медианой
for i in numeric_features:
    full_data[i].fillna(full_data[i].median(), inplace=True)

In [10]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
Alley            2919 non-null object
BedroomAbvGr     2919 non-null int64
BldgType         2919 non-null object
BsmtCond         2837 non-null object
BsmtExposure     2837 non-null object
BsmtFinSF1       2919 non-null float64
BsmtFinSF2       2919 non-null float64
BsmtFinType1     2840 non-null object
BsmtFinType2     2839 non-null object
BsmtFullBath     2919 non-null float64
BsmtHalfBath     2919 non-null float64
BsmtQual         2838 non-null object
BsmtUnfSF        2919 non-null float64
CentralAir       2919 non-null int64
Condition1       2919 non-null object
Condition2       2919 non-null object
Electrical       2919 non-null object
EnclosedPorch    2919 non-null int64
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      291

In [11]:
# кодируем переменные типа объекта
full_data = pd.get_dummies(full_data, columns=categorial_features)

X_train, X_test = full_data.iloc[:len(train_data), :], full_data.iloc[len(train_data):, :]

X_train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,...,0,0,0,0,0,1,0,1,1,0
1,1262,0,0,3,978.0,0.0,0.0,1.0,284.0,0,...,0,0,0,0,0,1,0,1,1,0
2,920,866,0,3,486.0,0.0,1.0,0.0,434.0,0,...,0,0,0,0,0,1,0,1,1,0
3,961,756,0,3,216.0,0.0,1.0,0.0,540.0,272,...,0,0,0,0,0,1,0,1,1,0
4,1145,1053,0,4,655.0,0.0,1.0,0.0,490.0,0,...,0,0,0,0,0,1,0,1,1,0


In [12]:
X_train.shape

(1460, 291)

#### Снижение размерности?
#### Какая задача - регрессия или классификация?

In [13]:
# построим самый простой регрессор
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, StratifiedKFold

clf = LinearRegression()

#### Регуляризация?
#### Оценка модели? 

In [14]:
y_train = train_data["SalePrice"]
X_train.drop("SalePrice",axis=1, inplace=True)

In [15]:
y_train.shape

(1460,)

In [16]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

-3906479513.909408
-454583368520.2452
-82932810.04720478
-700257336.3159174
-1948679884688.8848


In [17]:
from sklearn.preprocessing import StandardScaler

sclr= StandardScaler()
X_cols= X_train.columns
X = pd.DataFrame(sclr.fit_transform(X_train), columns = X_cols)


y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

-4.909425064695963e+25
-1.2983469194841347e+24
-5.056252418828677e+20
-1.081221949316388e+24
-6.464720677506622e+23


In [18]:
cross_val_score(clf, X_train, y_train, cv=5)

array([-1.90270250e+23, -6.43043934e+23, -6.36986456e+23, -5.69246216e+22,
       -3.28300667e+22])

In [19]:
X_train_short= X_train[['Id', 'MSSubClass', 'LotArea', 'OverallQual',\
      'OverallCond', 'YearBuilt', 'YearRemodAdd',\
      'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',\
     'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',\
     'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',\
     'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',\
     'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',\
     'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']]

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

params = {
    "n_estimators": [10, 50],
    "max_features": ["auto", "sqrt"],
    "max_depth": [5,10]
}
gs = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring="neg_mean_squared_error")
gs.fit(X_train, y_train)

gs.best_params_

{'max_depth': 10, 'max_features': 'auto', 'n_estimators': 50}

In [21]:
model = RandomForestRegressor(**gs.best_params_)
cross_val_score(model, X_train, y_train, cv=5)

array([0.8532122 , 0.90690289, 0.7989545 , 0.86788685, 0.86607345])

In [163]:
X = X_train
y = y_train
skf = StratifiedKFold(n_splits=5, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    scores.append(model.score(X_test, y_test))

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.7302422837178258
0.8668866786086367
0.8106396054183895
0.7462901243057354
0.6278869572694543


## Catboost?

## Titanic dataset 

In [22]:
train_df = pd.read_csv('titanic/train.csv')
test_df = pd.read_csv('titanic/test.csv')

In [23]:
train_df["Male"] = train_df.Sex.map({"male": 1, "female": 0}).astype(int) # альтернатива OneHotEncoding
test_df["Male"] = test_df.Sex.map({"male": 1, "female": 0}).astype(int)

train_df = train_df.drop(["PassengerId", "Ticket", "Cabin", "Name", "Sex", "SibSp", "Parch", "Embarked", "Fare", 'Age'], axis=1)
test_df = test_df.drop(["Ticket", "Cabin", "Name", "Sex", "SibSp", "Parch", "Embarked", "Fare",'Age'], axis=1)

X = train_df.drop("Survived", axis=1).fillna(np.nan)
y = train_df.Survived
X_test = test_df.drop("PassengerId", axis=1).fillna(np.nan)


X.tail()

,Pclass,Male
886,2,1
887,1,0
888,3,0
889,1,1
890,3,1


In [24]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()

In [25]:
skf = StratifiedKFold(n_splits=10, random_state=42)
skf.get_n_splits(X, y)

scores=[]

for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    scores.append(clf.score(X_test, y_test))

0.8111111111111111
0.7888888888888889
0.7752808988764045
0.8426966292134831
0.797752808988764
0.7752808988764045
0.7640449438202247
0.7415730337078652
0.8089887640449438
0.7613636363636364


In [31]:
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [32]:
clf = Pipeline([#('selectk', SelectKBest(k=6)), 
                     ('scaler', StandardScaler()),
                ('decision_tree',  RandomForestClassifier(n_estimators=25, max_depth=3))])

cross_val_score(estimator=clf, X=X, y=y, cv=5).mean()

0.775541840571646

### Метрики: оценки качества

### Практикум по *.Git системам:

!['git'](https://i1.wp.com/lanziani.com/slides/gitflow/images/gitflow_1.png?zoom=2)

1. Установка: https://git-scm.com/downloads
2. GUI: https://desktop.github.com/
3. Туториал: https://learngitbranching.js.org/